## LogisticRegression, Naive Bayesian Classifier, and kNN Implementation

In [1]:
import numpy as np

In [2]:
# Define logistic function
def logistic_func(x, theta):
    # compute z
    z = np.dot(x, theta)
    logistic_value = 1.0 / (np.exp(-z) + 1)
    return logistic_value

# Define gradient descent function
# alpha is importance factors, set it to ones when all data points are equally important
def log_gradient(theta, x, y, alpha):
    logistic_value = logistic_func(x, theta)
    gradient = np.dot(np.squeeze(alpha) * np.transpose(x),  logistic_value - y)
    return gradient

# Define the cost function
# alpha is importance factors, set it to ones when all data points are equally important
def cost_func(theta, x, y, alpha):
    log_func_v = logistic_func(x,theta)
    step1 = y * np.log(log_func_v)
    step2 = (1-y) * np.log(1 - log_func_v)
    final = (-step1 - step2) * alpha
    return np.mean(final)

In [3]:
X = np.array([[0, 1, 0], [1, 0, 0], [0, 0, 1], [1, 1, 1], [1, 1, 0]])
X

array([[0, 1, 0],
       [1, 0, 0],
       [0, 0, 1],
       [1, 1, 1],
       [1, 1, 0]])

In [4]:
y = np.array([[1],[1],[1],[0],[0]])
y

array([[1],
       [1],
       [1],
       [0],
       [0]])

In [5]:
# Insert ones into the first column of X to include constant term b in the formula
X_b = np.insert(X, [0], np.ones((5, 1)), axis=1)
X_b

array([[1, 0, 1, 0],
       [1, 1, 0, 0],
       [1, 0, 0, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 0]])

In [6]:
# Define gradient descent process
def gradient_descent(theta, x, y, alpha, lc=0.001, stoppingCost=0.1, epsilon=0.001):
    # number of iteration
    i=0
    # compute initial cost value
    pre_cost = cost_func(theta, x, y, alpha) 
    print('initial stage: iteration=',i,' w1=',theta[1, 0],' w2=',theta[2, 0],' w3=',theta[3, 0],' b=',theta[0, 0], ' cost=', pre_cost)
    # stopping condition: the cost is close to the stoppingCost
    while abs(pre_cost-stoppingCost) > epsilon:
        i = i + 1
        # compute derivatives
        d_theta = log_gradient(theta, x, y, alpha)
        # update theta
        theta = theta - lc * d_theta
        # compute current cost
        current_cost = cost_func(theta, x, y, alpha)
        pre_cost = current_cost
    print('final stage: iteration=',i,' w1=',theta[1, 0],' w2=',theta[2, 0],' w3=',theta[3, 0],' b=',theta[0, 0], ' cost=', pre_cost)

### Three different initial points

In [7]:
alpha = np.ones((5, 1))
theta = np.random.random_sample((4, 1))
gradient_descent(theta, X_b, y, alpha)

initial stage: iteration= 0  w1= 0.024406095618  w2= 0.273502828378  w3= 0.976566705746  b= 0.968976960363  cost= 0.912319176777
final stage: iteration= 26483  w1= -3.62344159872  w2= -3.62103338054  w3= -1.37407582517  b= 5.4995054592  cost= 0.100999218101


In [8]:
theta = np.random.random_sample((4, 1))
gradient_descent(theta, X_b, y, alpha)

initial stage: iteration= 0  w1= 0.99393060029  w2= 0.915542728603  w3= 0.988356863571  b= 0.69661849635  cost= 1.3640223728
final stage: iteration= 27667  w1= -3.62044559615  w2= -3.62102454028  w3= -1.38347433045  b= 5.49872340188  cost= 0.100999118443


In [9]:
theta = np.random.random_sample((4, 1))
gradient_descent(theta, X_b, y, alpha)

initial stage: iteration= 0  w1= 0.209748854756  w2= 0.29612039617  w3= 0.345735839439  b= 0.455795664615  cost= 0.800768871087
final stage: iteration= 26854  w1= -3.61986312752  w2= -3.61908924796  w3= -1.3914751938  b= 5.49808331879  cost= 0.100998830289


### Three different initial points with importance factors

In [10]:
# Importance factors
alpha = np.array([[2],[2],[2],[3],[3]])
theta = np.random.random_sample((4, 1))
gradient_descent(theta, X_b, y, alpha)

initial stage: iteration= 0  w1= 0.309876956707  w2= 0.732881480595  w3= 0.0339833709998  b= 0.637105926026  cost= 2.62501548653
final stage: iteration= 30643  w1= -5.35113029025  w2= -5.35097313351  w3= -2.02518692491  b= 7.92457872494  cost= 0.100998462143


In [11]:
theta = np.random.random_sample((4, 1))
gradient_descent(theta, X_b, y, alpha)

initial stage: iteration= 0  w1= 0.418558531706  w2= 0.871097414493  w3= 0.538385337814  b= 0.789539332478  cost= 3.19920541762
final stage: iteration= 30726  w1= -5.35154443579  w2= -5.35138400404  w3= -2.02143423084  b= 7.92491842444  cost= 0.100997063566


In [12]:
theta = np.random.random_sample((4, 1))
gradient_descent(theta, X_b, y, alpha)

initial stage: iteration= 0  w1= 0.810785041115  w2= 0.676980136432  w3= 0.331019055677  b= 0.934262425502  cost= 3.42973928161
final stage: iteration= 30676  w1= -5.35118272205  w2= -5.35123185163  w3= -2.02372008151  b= 7.92470310257  cost= 0.100998187549


### We can see that in both cases the final weights and cost converge to the same values (within epsilon range)

## Binary Classifier

In [13]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris

data = load_iris()
X = data.data
y = data.target

### One vs Others

In [14]:
def train_one_vs_others(X, y):
    # get all unique classes
    classes = list(set(y))
    # number of classes
    nclasses = len(classes)
    # create a model list
    models = []
    # loop through all classes and create different one vs others models
    for i in range(nclasses):
        # get binary labels (one vs others)
        y_bi = np.array([1 if label == classes[i] else 0 for label in y])
        # train Logistic Regression model
        from sklearn import linear_model
        clf = linear_model.LogisticRegression()
        clf.fit(X, y_bi)
        models.append(clf)
    return classes, models

def predict_one_vs_others(X, models, classes):
    prediction_list = []
    # go through all the models
    for model in models:
        # get the probability of predicting class 1, 
        # which means how likely the current sample is to be the class 1 of the current model 
        # and convert it to a column vector 
        prediction = model.predict_proba(X)[:, -1].reshape(-1, 1)
        prediction_list.append(prediction)
    # combine all predictions from different classifiers
    pred_proba = np.hstack(prediction_list)
    # choose the one with the highest prediction probability as the final prediction class
    predictions_numeric = np.argmax(pred_proba, axis=1)
    # convert back to original class labels
    predictions = np.array([classes[i] for i in predictions_numeric])
    return predictions

### Test the implementation

In [15]:
from sklearn.model_selection import StratifiedKFold
kf = StratifiedKFold(n_splits=5)

for train_index, test_index in kf.split(X,y):
    X_train=X[train_index]
    y_train=y[train_index]
    X_test=X[test_index]
    y_test=y[test_index]
    
    classes, models = train_one_vs_others(X_train, y_train)
    predictions = predict_one_vs_others(X_test, models, classes)
    
    print('accuracy of one vs others logistic regression=',np.mean(predictions == y_test))

accuracy of one vs others logistic regression= 1.0
accuracy of one vs others logistic regression= 0.966666666667
accuracy of one vs others logistic regression= 0.933333333333
accuracy of one vs others logistic regression= 0.9
accuracy of one vs others logistic regression= 1.0


### One vs One

In [16]:
def train_one_vs_one(X, y):
    # get all unique classes
    classes = list(set(y))
    # get all one vs one pair in the classes
    import itertools
    pairs = list(itertools.combinations(classes, 2))
    # number of classes
    nclasses = len(classes)
    # create a model list
    models = []
    # loop through all pairs and create different one vs one models
    for pair in pairs:
        # get one vs one data samples (the samples with labels contained in the pair)
        train_X = X[(y == pair[0]) | (y == pair[1])]
        train_y = y[(y == pair[0]) | (y == pair[1])]
        # convert label to binary label to train
        # pair[0] is 0, pair[1] is 1
        y_bi = np.array([0 if label == pair[0] else 1 for label in train_y])
        # train Logistic Regression model
        from sklearn import linear_model
        clf = linear_model.LogisticRegression()
        clf.fit(train_X, y_bi)
        models.append(clf)
    return pairs, models

def predict_one_vs_one(X, models, pairs):
    prediction_list = []
    # nmodels must be equal to npairs
    nmodels = len(models)
    # go through all the models
    for i in range(nmodels):
        # get class pair and model
        pair = pairs[i]
        model = models[i]
        prediction_bi = model.predict(X)
        # convert back to original class in order to do a majority vote
        prediction = np.array([pair[label] for label in prediction_bi]).reshape(-1, 1)
        prediction_list.append(prediction)
    # combine all predictions from different classifiers
    pred_classes = np.hstack(prediction_list)
    # choose the one with the highest count as the final prediction class
    from collections import Counter
    predictions = np.array([Counter(pred).most_common(1)[0][0] for pred in pred_classes])
    return predictions

### Test the implementation

In [17]:
kf = StratifiedKFold(n_splits=5)

for train_index, test_index in kf.split(X,y):
    X_train=X[train_index]
    y_train=y[train_index]
    X_test=X[test_index]
    y_test=y[test_index]
    
    pairs, models = train_one_vs_one(X_train, y_train)
    predictions = predict_one_vs_one(X_test, models, pairs)
    
    print('accuracy of one vs others logistic regression=',np.mean(predictions == y_test))

accuracy of one vs others logistic regression= 1.0
accuracy of one vs others logistic regression= 1.0
accuracy of one vs others logistic regression= 0.933333333333
accuracy of one vs others logistic regression= 0.966666666667
accuracy of one vs others logistic regression= 1.0


Both procedures have high accuracy

## Naive Bayesian Classifier

In [18]:
def compute_join_probability(X_train, y_train, X_test, h_test):
    likelihood = 1
    # go through all features in the sample X_test
    for i in range(len(X_test)):
        # get the ith column in X_train
        train_feature = X_train[:, i]
        # get required feature value
        test_feature = X_test[i]
        # compute probability of p(x | y) 
        count_x_y = len(X_train[(train_feature == test_feature) & (y_train == h_test)])
        count_y = len(y_train[y_train == h_test])
        p_x_y = count_x_y / count_y
        # update likelihood
        likelihood = likelihood * p_x_y
    # compute the probability of y == h_test
    p_y = len(y_train[y_train == h_test]) / len(y_train)
    # multiply the likelihood by the probability of y == h_test
    likelihood = likelihood * p_y
    return likelihood

def predict_naive_bayes_classifier(X_train, y_train, X_test):
    h_0 = compute_join_probability(X_train, y_train, X_test, 0)
    h_1 = compute_join_probability(X_train, y_train, X_test, 1)
    prediction = 0
    # compare two hypothesis
    if h_1 > h_0:
        prediction = 1
    return prediction

In [19]:
X_train = np.array([[0, 1, 0], [1, 0, 0], [0, 0, 1], [1, 1, 1], [1, 1, 0]])
y_train = np.array([1, 1, 1, 0, 0])

X_test1 = np.array([1, 0, 0])
X_test2 = np.array([1, 0, 1])

prediction = predict_naive_bayes_classifier(X_train, y_train, X_test1)
print("the prediction of data ", X_test1, "is ", prediction, "the actual label is ", 0)

prediction = predict_naive_bayes_classifier(X_train, y_train, X_test2)
print("the prediction of data ", X_test2, "is ", prediction, "the actual label is ", 1)

the prediction of data  [1 0 0] is  1 the actual label is  0
the prediction of data  [1 0 1] is  1 the actual label is  1


## kNN

In [20]:
# Define a function to compute the Euclidean distance between each training sample with a given test sample
def compute_Euclidean_distance(X_train, X_test):
    import numpy as np
    return np.linalg.norm(X_train - X_test, axis=1)

In [21]:
def predict_kNN(X_train, y_train, X_test, k, distance_metric=compute_Euclidean_distance):
    # compute the distances
    distances = distance_metric(X_train, X_test)
    # get the indices of the data which have the first k shortest distance from X_test
    k_nearest_indices = np.argsort(distances)[:k]
    # get the corresponding labels
    k_nearest_labels = y_train[k_nearest_indices]
    # do a majority vote
    from collections import Counter
    return Counter(k_nearest_labels).most_common(1)[0][0]

### Test the implementation

In [22]:
X_train = np.array([[0, 1, 0], [1, 0, 0], [0, 0, 1], [1, 1, 1], [1, 1, 0]])
y_train = np.array([1, 1, 1, 0, 0])

X_test1 = np.array([1, 0, 0])
X_test2 = np.array([1, 0, 1])
y_test1 = 0
y_test2 = 1

# for each odd number of k less than the sample size
# do prediction
for k in range(1, len(X_train) + 1, 2):
    pred1 = predict_kNN(X_train, y_train, X_test1, k)
    pred2 = predict_kNN(X_train, y_train, X_test2, k)
    print("by using norm-2 distance, the ", k, "- NN model predictions are: ", pred1, ",",pred2)
    # compute accuracy
    accuracy = np.mean([pred1==y_test1, pred2==y_test2])
    print("by using norm-2 distance, the ", k, "- NN model accuracy is: ", accuracy)

by using norm-2 distance, the  1 - NN model predictions are:  1 , 1
by using norm-2 distance, the  1 - NN model accuracy is:  0.5
by using norm-2 distance, the  3 - NN model predictions are:  1 , 1
by using norm-2 distance, the  3 - NN model accuracy is:  0.5
by using norm-2 distance, the  5 - NN model predictions are:  1 , 1
by using norm-2 distance, the  5 - NN model accuracy is:  0.5
